## Optuna를 활용한 CNN Model의 Hyperparameter Tuning
- https://colab.research.google.com/drive/1TEILbIeyE1wW3daNWOx9rLq0Hc9TXmbV#scrollTo=Bm7t3gfI-1GF

In [71]:
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

In [72]:
# GPU 정보도 받으면서 GPU 설정하기
print("============================================================================================")
# set device to cpu or cuda
device = torch.device('cpu')
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
print("============================================================================================")

Device set to : NVIDIA GeForce GTX 1050 Ti


## 데이터

In [73]:
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [74]:
print(train_data)
print()
print(test_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [75]:
def get_mnist_loaders(train_batch_size, test_batch_size):
    
    train_loader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False)
    
    return train_loader, test_loader

## CNN

In [76]:
class Net(nn.Module):
    def __init__(self, out_channels1, out_channels2, prob1, prob2, prob3, hidden1):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
                                    nn.Conv2d(1, out_channels1, kernel_size=3, padding=1),  # in_channels는 절대 건들면 안 됨
                                    nn.ReLU(),
                                    nn.MaxPool2d(2),
                                    nn.Dropout2d(prob1)  )

        self.layer2 = nn.Sequential(
                                    nn.Conv2d(out_channels1, out_channels2, kernel_size=3, padding=1), # 
                                    nn.ReLU(),
                                    nn.MaxPool2d(2),
                                    nn.Dropout2d(prob2) )
        
        self.linear1 = nn.Linear(out_channels2 * 7 * 7, hidden1) # out_channels2 * 7 * 7은 CNN에서 output_size 구하는 계산 식에서 얻음
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(prob3)
        self.linear2 = nn.Linear(hidden1, 10)
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)

        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.logsoftmax(x)

        return x

In [77]:
a = torch.ones(1, 1, 28, 28)
print(a.shape)

torch.Size([1, 1, 28, 28])


## TRAIN
- nll_loss 사용해서 loss가 음수로 출력됨

In [78]:
def train(log_interval, model, train_loader, optimizer, epoch, loss_func):
    model.train()

    for batch_idx, (x, y_true) in enumerate(train_loader):

        x, y_true = x.to(device), y_true.to(device)#🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨

        optimizer.zero_grad()
        y_pred = model(x) ##🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨
        loss = loss_func(y_pred, y_true) ##🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print( f'Train Epoch: {epoch} [{batch_idx*len(x)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}' )

## TEST

In [79]:
def test(model, test_loader, loss_func):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y_true in test_loader:

            x, y_true = x.to(device), y_true.to(device)

            y_pred = model(x) # torch.Size([1000, 10])

            # loss_func(y_pred, y_true).item() ====> Scalar
            test_loss += loss_func(y_pred, y_true).item()  # loss에 대한 스칼라 값 얻음

            # dimension을 유지한 채로 argmax를 찾음
            pred = y_pred.argmax(dim=1, keepdim=True)

            # 그게 정답과 같은지 비교(.eq)해줌 ==> 결과는 True/False로 나옴
            # True인 것들을 합산하면, 제대로 예측한 개수를 얻을 수 있음
            correct += pred.eq(y_true.view_as(pred)).sum().item() 

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print( f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n' )
    
    return accuracy

In [80]:
# 잘되는지 테스트 하는 곳
'''
net = Net(out_channels1=32, out_channels2=64, prob1=0.2, prob2=0.2, prob3=0.2, hidden1=128)
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train_loader, test_loader = get_mnist_loaders(train_batch_size=64, test_batch_size=1000)

for epoch in range(1, 10+1):
    train(log_interval=100, model=net, train_loader=train_loader, optimizer=optimizer, epoch=epoch, loss_func=criterion)
    test(model=net, test_loader=test_loader, loss_func=criterion)
'''

'\nnet = Net(out_channels1=32, out_channels2=64, prob1=0.2, prob2=0.2, prob3=0.2, hidden1=128)\noptimizer = optim.Adam(net.parameters(), lr=0.001)\ncriterion = nn.CrossEntropyLoss()\ntrain_loader, test_loader = get_mnist_loaders(train_batch_size=64, test_batch_size=1000)\n\nfor epoch in range(1, 10+1):\n    train(log_interval=100, model=net, train_loader=train_loader, optimizer=optimizer, epoch=epoch, loss_func=criterion)\n    test(model=net, test_loader=test_loader, loss_func=criterion)\n'

## 본격적 학습

In [81]:
def train_mnist(trial):

    cfg = { #'device' : "cuda" if torch.cuda.is_available() else "cpu",##🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨🟨
            'train_batch_size' : trial.suggest_categorical('train_batch_size', [64, 128, 256]),
            'test_batch_size' : trial.suggest_categorical('test_batch_size', [64, 128, 256]),
            'n_epochs' : 5,
            'seed' : 0,
            'log_interval' : 100, # 100번째 배치마다 loss 출력할 거임(로그 남기는 거)
            'save_model' : False,
            'lr' : trial.suggest_loguniform('lr', 1e-3, 1e-2),          
            'momentum': trial.suggest_uniform('momentum', 0.4, 0.99),
            'optimizer': trial.suggest_categorical('optimizer',['RAdam', 'Adam', 'RMSProp']),

            'out_channels1': trial.suggest_categorical('out_channels1', [16, 32, 64]),
            'out_channels2': trial.suggest_categorical('out_channels2', [16, 32, 64]),
            'prob1': trial.suggest_uniform('prob1', 0.2, 0.5),
            'prob2': trial.suggest_uniform('prob2', 0.2, 0.5),
            'prob3': trial.suggest_uniform('prob3', 0.2, 0.5),
            'hidden1': trial.suggest_categorical('hidden1', [64, 128, 256]),
            }

    # suggest_categorical에서 사용할 수 있는 건 int, str, bool, ......뿐임. 아래와 같이 작성해야 경고 문구 안 나옴.
    optimizer_candidate = {'RAdam':optim.RAdam, 'Adam':optim.Adam, 'RMSProp':optim.RMSprop}[cfg['optimizer']]


    #torch.manual_seed(cfg['seed'])
    model = Net(cfg['out_channels1'], cfg['out_channels2'], cfg['prob1'], cfg['prob2'], cfg['prob3'], cfg['hidden1']).to(device)
    optimizer = optimizer_candidate(model.parameters(), lr=cfg['lr'])
    loss_func = nn.NLLLoss().to(device)
    train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])


    for epoch in range(1, cfg['n_epochs'] + 1):
        train(cfg['log_interval'], model, train_loader, optimizer, epoch, loss_func)
        test_accuracy = test(model, test_loader, loss_func)
        # print(test_accuracy)
        
    if cfg['save_model']:
        torch.save(model.state_dict(), "mnist_cnn.pt")

    return test_accuracy

## Optimization

In [82]:
sampler = optuna.samplers.TPESampler()
    
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=train_mnist, n_trials=20)

[I 2022-08-08 20:00:36,888] A new study created in memory with name: no-name-73ea5c4d-6872-466e-93e6-4a697e703441
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.radam.RAdam'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.rmsprop.RMSprop'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310981
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.472257
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.247451
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.289236
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.110843

Test set: Average loss: 0.0003, Accuracy: 9726/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.157784
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.128116
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.069859
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.070378
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.026730

Test set: Average loss: 0.0002, Accuracy: 9827/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.114039
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.060000
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.136222
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.048388
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.060345

Test set: Average loss: 0.0002, Accuracy: 9843/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.099550
Train Epoch: 4 [12800/60000 (21%)]	Lo

[I 2022-08-08 20:01:18,555] Trial 0 finished with value: 99.02 and parameters: {'train_batch_size': 128, 'test_batch_size': 256, 'lr': 0.003038332838428516, 'momentum': 0.7675489001374012, 'optimizer': <class 'torch.optim.radam.RAdam'>, 'out_channels1': 16, 'out_channels2': 32, 'prob1': 0.21765708977921416, 'prob2': 0.26984438461641297, 'prob3': 0.31985142559466134, 'hidden1': 256}. Best is trial 0 with value: 99.02.



Test set: Average loss: 0.0001, Accuracy: 9902/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305410


c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.radam.RAdam'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.rmsprop.RMSprop'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)


...


Test set: Average loss: 0.0001, Accuracy: 9891/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.287674


c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.radam.RAdam'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.rmsprop.RMSprop'> which is of type type.
  warnings.warn(message)
c:\Anaconda3\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.306085
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.068973
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.929637
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.646700
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.582259
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.731235
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.572772
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.354388
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.516819

Test set: Average loss: 0.0006, Accuracy: 9534/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.370966
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.573470
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.302737
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.319409
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.401838
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.529979
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.139972
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.244475
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.182288
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.167

[I 2022-08-08 20:14:31,461] Trial 19 finished with value: 98.3 and parameters: {'train_batch_size': 64, 'test_batch_size': 256, 'lr': 0.0036151216702108332, 'momentum': 0.7828590183994253, 'optimizer': <class 'torch.optim.rmsprop.RMSprop'>, 'out_channels1': 64, 'out_channels2': 16, 'prob1': 0.3057144797270942, 'prob2': 0.3832311358153075, 'prob3': 0.3431025121965455, 'hidden1': 64}. Best is trial 0 with value: 99.02.



Test set: Average loss: 0.0002, Accuracy: 9830/10000 (98%)



## The Best Trial 확인

In [84]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

train_batch_size: 128
test_batch_size: 256
lr: 0.003038332838428516
momentum: 0.7675489001374012
optimizer: <class 'torch.optim.radam.RAdam'>
out_channels1: 16
out_channels2: 32
prob1: 0.21765708977921416
prob2: 0.26984438461641297
prob3: 0.31985142559466134
hidden1: 256


In [89]:
optuna.visualization.plot_parallel_coordinate(study,params=['lr','momentum'])

## The End